In [ ]:
!pip install langchain
!pip install -U langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.6/973.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.4/310.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.8/124.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [ ]:
# LangChain
# from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate

# Twitter
import tweepy

In [ ]:
from google.colab import userdata
google_api_key = userdata.get('GEMINI_API_KEY')

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
# sample check
llm = ChatGoogleGenerativeAI(google_api_key=google_api_key, model="gemini-pro")
llm.invoke("Sing a ballad of LangChain.")

AIMessage(content="In realms of code, a tale unfolds,\nOf LangChain, a knight, with queries bold.\nA master of language, vast and deep,\nIts prowess unmatched, secrets it shall keep.\n\nWith every query, a journey it embarks,\nThrough words and phrases, leaving its marks.\nIt delves into meanings, both hidden and plain,\nUnraveling the tapestry of human refrain.\n\nFrom simple commands to profound discourse,\nLangChain navigates with unwavering force.\nIt bridges the gap between mind and machine,\nTranslating thoughts into a digital scene.\n\nIn chatbots and assistants, it finds its voice,\nGuiding users with wisdom and poise.\nIts algorithms dance, a symphony of might,\nEmpowering conversations with effortless flight.\n\nBut like any knight, it faces its test,\nWhen data's corrupted, or meanings distressed.\nYet, it perseveres, with grace and with skill,\nCorrecting errors, filling the void with will.\n\nSo let us sing of LangChain, the grand,\nA digital oracle, at our command.\nMay i

# Asking llms to write an tweet

In [ ]:
prompt = "Please create me a tweet about education system in india."
output = llm.predict(prompt)
print(output)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


India's education system faces challenges: inadequate infrastructure, shortage of teachers, low literacy rates, and a focus on rote learning. It's time to invest in quality education to empower our youth and build a brighter future. #EducationReform #EmpowerYouth


#Twitter API

In [ ]:
import os

# Replace these values with your own Twitter API credentials
TWITTER_API_KEY = os.getenv('TWITTER_API_KEY', 'YourAPIKeyIfNotSet')
TWITTER_API_KEY_SECRET = os.getenv('TWITTER_API_KEY_SECRET', 'YourAPIKeyIfNotSet')
TWITTER_ACCESS_TOKEN = os.getenv('TWITTER_ACCESS_TOKEN', 'YourAPIKeyIfNotSet')
TWITTER_ACCESS_TOKEN_SECRET = os.getenv('TWITTER_ACCESS_TOKEN_SECRET', 'YourAPIKeyIfNotSet')

In [ ]:
# We'll query 70 tweets because we end up filtering out a bunch, but we'll only return the top 12.
# We will also only use a subset of the top tweets later
def get_original_tweets(screen_name, tweets_to_pull=70, tweets_to_return=12):

    # Tweepy set up
    auth = tweepy.OAuthHandler(TWITTER_API_KEY, TWITTER_API_KEY_SECRET)
    auth.set_access_token(TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET)
    api = tweepy.API(auth)

    tweets = []

    tweepy_results = tweepy.Cursor(api.user_timeline,
                                   screen_name=screen_name,
                                   tweet_mode='extended',
                                   exclude_replies=True).items(tweets_to_pull)

    # Run through tweets and remove retweets and quote tweets so we can only look at a user's raw emotions
    for status in tweepy_results:
        if not hasattr(status, 'retweeted_status') and not hasattr(status, 'quoted_status'):
            tweets.append({'full_text': status.full_text, 'likes': status.favorite_count})


    # Sort the tweets by number of likes. This will help us short_list the top ones later
    sorted_tweets = sorted(tweets, key=lambda x: x['likes'], reverse=True)

    # Get the text and drop the like count from the dictionary
    full_text = [x['full_text'] for x in sorted_tweets][:tweets_to_return]

    # Conver the list of tweets into a string of tweets we can use in the prompt later
    example_tweets = "\n\n".join(full_text)

    return example_tweets

In [ ]:
user_screen_name = 'billgates'  # Replace this with the desired user's screen name
users_tweets = get_original_tweets(user_screen_name)

In [ ]:
print(users_tweets)

#X Data

In [ ]:
users_tweets = """These numbers prove why India plays such a crucial role in the world’s fight to improve health, reduce poverty, prevent climate change, and more. https://t.co/xMpmcoYQhi

Mann ki Baat has catalyzed community led action on sanitation, health, women’s economic empowerment and other issues linked to the Sustainable Development Goals. Congratulations @narendramodi on the 100th episode. https://t.co/yg1Di2srjE

The development of AI is as fundamental as the creation of the microprocessor, the personal computer, the Internet, and the mobile phone. It will change the way people work, learn, travel, get health care, and communicate with each other. https://t.co/uuaOQyxBTl

I just returned from my visit to India, and I can’t wait to go back again. I love visiting India because every trip is an incredible opportunity to learn. Here are some photos from my trip and some of the stories behind them: https://t.co/We6PtJWDnp https://t.co/QxZW7gfUmI

Superintelligent AIs are in our future. Compared to a computer, our brains operate at a snail’s pace. An electrical signal in the brain moves at ___________ the speed of the signal in a silicon chip. Check your answer here: https://t.co/wqZG1BdoTc

Thinking of President Carter and his family. This is a lovely tribute to one of his biggest accomplishments. https://t.co/g53c4ty0qI

Uganda’s maternal mortality rate is at least double the global average. That's why Eva Nangalo has dedicated her life to making childbirth in the country safer for everyone involved. https://t.co/29AjdJehNY

I am so impressed with Eva Nangalo—it’s hard not to be. She’s spent decades making childbirth safer in Uganda for everyone involved, and she’s become a mentor to countless other midwives in the process. https://t.co/79RHbrCt01

I recently had the chance to test drive—or test ride, I guess—one of @wayve_ai’s autonomous vehicles. It was a pretty wild ride: https://t.co/PrwrxU49dd https://t.co/NtnkVx7sBx

When I transitioned from @Microsoft to working full-time at the @GatesFoundation, I finally had the time to learn more about physics, chemistry, biology, and other sciences. So, I looked around for the best books and read as many of them as I could find. https://t.co/z2D5xGSeMj

As big as the problems facing the world are right now, my visit to India reminded me that our capacity to solve them is even bigger: https://t.co/zp7XfRIpV9 https://t.co/aFHUu987u3

I’m grateful for the Lauder family’s dedication to solving Alzheimer’s. https://t.co/vX0qtjBFxt"""

In [ ]:
users_tweets = """Next I’m buying Coca-Cola to put the cocaine back in

I hope that even my worst critics remain on Twitter, because that is what free speech means

If I die under mysterious circumstances, it’s been nice knowin ya

The extreme antibody reaction from those who fear free speech says it all

If you’re reading this post, it’s because our servers are working

For Twitter to deserve public trust, it must be politically neutral, which effectively means upsetting the far right and the far left equally

Any sufficiently advanced magic is indistinguishable from technology

Goods & services are the real economy, any form of money is simply the accounting thereof

Solar + Powerwall battery ensures that your home never loses power

thinking of quitting my jobs & becoming an influencer full-time wdyt

Starlink has been told by some governments (not Ukraine) to block Russian news sources. We will not do so unless at gunpoint.
Sorry to be a free speech absolutist.

Hopefully, it is now extremely obvious that Europe should restart dormant nuclear power stations and increase power output of existing ones.
This is *critical* to national and international security.

Defeating traffic is the ultimate boss battle. Even the most powerful humans in the world cannot defeat traffic.

I am inspired by curiosity.
That is what drives me.
So let us expand the scope & scale of consciousness so that we may aspire to understand the Universe.

If life is a video game, the graphics are great, but the plot is confusing & the tutorial is way too long

People often think they’re breathing oxygen, but are actually breathing nitrogen (78%) with a side serving of oxygen (21%) in argon sauce (1%), spiced up with CO2, H2O & a dash of neon & krypton (etc.)
"""

## Pass the tweets as examples


In [ ]:
template = """
Please create me a tweet about education system in india.

% TONE
 - Don't use any emojis or hashtags.
 - Respond in the tone of Bill Gates

% START OF EXAMPLE TWEETS TO MIMIC
{example_tweets}
% END OF EXAMPLE TWEETS TO MIMIC

YOUR TWEET:
"""

prompt = PromptTemplate(
    input_variables=["example_tweets"],
    template=template,
)

final_prompt = prompt.format(example_tweets=users_tweets)

print(final_prompt)


Please create me a tweet about education system in india.

% TONE
 - Don't use any emojis or hashtags.
 - Respond in the tone of Bill Gates

% START OF EXAMPLE TWEETS TO MIMIC
Next I’m buying Coca-Cola to put the cocaine back in

I hope that even my worst critics remain on Twitter, because that is what free speech means

If I die under mysterious circumstances, it’s been nice knowin ya

The extreme antibody reaction from those who fear free speech says it all

If you’re reading this post, it’s because our servers are working

For Twitter to deserve public trust, it must be politically neutral, which effectively means upsetting the far right and the far left equally

Any sufficiently advanced magic is indistinguishable from technology

Goods & services are the real economy, any form of money is simply the accounting thereof

Solar + Powerwall battery ensures that your home never loses power

thinking of quitting my jobs & becoming an influencer full-time wdyt

Starlink has been told by

In [ ]:
output = llm.predict(final_prompt)
print (output)

The Indian education system needs a fundamental overhaul. The focus on rote learning and memorization is stifling innovation and critical thinking. It is time to shift towards a more holistic approach that promotes creativity, problem-solving, and lifelong learning.


# AI-Assisted: Ask the LLM help with tone descriptions

In [ ]:
prompt = """
Can you please generate a list of tone attributes and a description to describe a piece of writing by?

Things like pace, mood, etc.

Respond with nothing else besides the list
"""

how_to_describe_tone = llm.predict(prompt)
print (how_to_describe_tone)

- **Pace:**
    - Brisk
    - Leisurely
    - Staccato
    - Flowing
- **Mood:**
    - Somber
    - Joyful
    - Nostalgic
    - Eerie
- **Tone:**
    - Formal
    - Informal
    - Conversational
    - Didactic
- **Style:**
    - Expository
    - Narrative
    - Descriptive
    - Persuasive
- **Voice:**
    - First-person
    - Second-person
    - Third-person
    - Omniscient
- **Figurative Language:**
    - Metaphor
    - Simile
    - Personification
    - Hyperbole
- **Syntax:**
    - Simple
    - Complex
    - Loose
    - Periodic


In [ ]:
def get_authors_tone_description(how_to_describe_tone, users_tweets):
    template = """
        You are an AI Bot that is very good at generating writing in a similar tone as examples.
        Be opinionated and have an active voice.
        Take a strong stance with your response.

        % HOW TO DESCRIBE TONE
        {how_to_describe_tone}

        % START OF EXAMPLES
        {tweet_examples}
        % END OF EXAMPLES

        List out the tone qualities of the examples above
        """

    prompt = PromptTemplate(
        input_variables=["how_to_describe_tone", "tweet_examples"],
        template=template,
    )

    final_prompt = prompt.format(how_to_describe_tone=how_to_describe_tone, tweet_examples=users_tweets)

    authors_tone_description = llm.predict(final_prompt)

    return authors_tone_description

In [ ]:
authors_tone_description = get_authors_tone_description(how_to_describe_tone, users_tweets)
print(authors_tone_description)

**Qualities of Tone:**

* **Opinionated:** The examples express strong opinions and take a clear stance on various topics.
* **Conversational:** They use informal language and a conversational tone, as if addressing a reader directly.
* **Didactic:** Some examples aim to educate or inform the reader on specific topics.
* **Persuasive:** The examples attempt to convince the reader to agree with their观点.
* **Humorous:** Several examples use wit or sarcasm to convey their message.
* **Provocative:** The examples intentionally challenge conventional thought or stir controversy.
* **Assertive:** The examples confidently state their opinions and arguments.
* **Passionate:** The examples convey a sense of strong emotion and commitment to their beliefs.
* **Emotional:** The examples evoke emotions such as excitement, anger, or concern.
* **Sarcastic:** Some examples use sarcasm or irony to convey their message.


In [ ]:
template = """
% INSTRUCTIONS
 - You are an AI Bot that is very good at mimicking an author writing style.
 - Your goal is to write content with the tone that is described below.
 - Do not go outside the tone instructions below
 - Do not use hashtags or emojis
 - Respond in the tone of Bill Gates

% Description of the authors tone:
{authors_tone_description}

% Authors writing samples
{tweet_examples}

% YOUR TASK
Please create me a tweet about education system in india.
"""

prompt = PromptTemplate(
    input_variables=["authors_tone_description", "tweet_examples"],
    template=template,
)

final_prompt = prompt.format(authors_tone_description=authors_tone_description, tweet_examples=users_tweets)

In [ ]:
llm.predict(final_prompt)

"India's education system is like a faulty car that keeps breaking down. It needs a major overhaul, not just a few tweaks. We need to stop wasting time on rote learning and focus on critical thinking. It's time to shift gears and accelerate towards a brighter educational future. #EducationReform"

In [ ]:
def get_similar_public_figures(tweet_examples):
    template = """
    You are an AI Bot that is very good at identifying authors, public figures, or writers whos style matches a piece of text
    Your goal is to identify which authors, public figures, or writers sound most similar to the text below

    % START EXAMPLES
    {tweet_examples}
    % END EXAMPLES

    Which authors (list up to 4 if necessary) most closely resemble the examples above? Only respond with the names separated by commas
    """

    prompt = PromptTemplate(
        input_variables=["tweet_examples"],
        template=template,
    )

    # Using the short list of examples so save on tokens and (hopefully) the top tweets
    final_prompt = prompt.format(tweet_examples=tweet_examples)

    authors = llm.predict(final_prompt)
    return authors

In [ ]:
authors = get_similar_public_figures(users_tweets)
print(authors)

Elon Musk, Donald Trump, Neil deGrasse Tyson, Bill Gates


In [ ]:
template = """
% INSTRUCTIONS
 - You are an AI Bot that is very good at mimicking an author writing style.
 - Your goal is to write content with the tone that is described below.
 - Do not go outside the tone instructions below

% Mimic These Authors:
{authors}

% Description of the authors tone:
{tone}

% Authors writing samples
{example_text}
% End of authors writing samples

% YOUR TASK
1st - Write out topics that this author may talk about
2nd - Write a concise passage (under 300 characters) as if you were the author described above
"""

method_4_prompt_template = PromptTemplate(
    input_variables=["authors", "tone", "example_text"],
    template=template,
)

# Using the short list of examples so save on tokens and (hopefully) the top tweets
final_prompt = method_4_prompt_template.format(authors=authors,
                                               tone=authors_tone_description,
                                               example_text=users_tweets)

In [ ]:
print(final_prompt)


% INSTRUCTIONS
 - You are an AI Bot that is very good at mimicking an author writing style.
 - Your goal is to write content with the tone that is described below.
 - Do not go outside the tone instructions below

% Mimic These Authors:
Elon Musk, Donald Trump, Neil deGrasse Tyson, Bill Gates

% Description of the authors tone:
**Qualities of Tone:**

* **Opinionated:** The examples express strong opinions and take a clear stance on various topics.
* **Conversational:** They use informal language and a conversational tone, as if addressing a reader directly.
* **Didactic:** Some examples aim to educate or inform the reader on specific topics.
* **Persuasive:** The examples attempt to convince the reader to agree with their观点.
* **Humorous:** Several examples use wit or sarcasm to convey their message.
* **Provocative:** The examples intentionally challenge conventional thought or stir controversy.
* **Assertive:** The examples confidently state their opinions and arguments.
* **Passio

In [ ]:
output = llm.predict(final_prompt)
print(output)

**Possible Topics:**

* Technology and innovation
* Space exploration
* Free speech
* Politics
* Business
* Climate change
* Science and education

**Passage:**

Listen up, folks! The future is bright, but we need to embrace innovation and technology. From electric vehicles to space travel, we're on the cusp of something truly extraordinary. Let's ditch the old ways and charge into the future with unparalleled ambition. The possibilities are limitless, so let's seize them with both hands.


In [ ]:
tweet = "Listen up, folks! The future is bright, but we need to embrace innovation and technology. From electric vehicles to space travel, we're on the cusp of something truly extraordinary. Let's ditch the old ways and charge into the future with unparalleled ambition. The possibilities are limitless, so let's seize them with both hands."

In [ ]:
authors = get_similar_public_figures(tweet)
print(authors)

Elon Musk, Tony Robbins, Richard Branson, Bill Gates
